<a href="https://colab.research.google.com/github/Saeedaaman/GB/blob/main/M10_L3_Azure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyze Text
The Language service is an Azure AI service that supports analysis of text, including language detection, sentiment analysis, key phrase extraction, and entity recognition.

For example, suppose a travel agency wants to process hotel reviews that have been submitted to the company’s web site. By using the Language service, they can determine the language each review is written in, the sentiment (positive, neutral, or negative) of the reviews, key phrases that might indicate the main topics discussed in the review, and named entities, such as places, landmarks, or people mentioned in the reviews.

Install the dependencies for text analytics. This contains the SDK to call the azure text-analytics API.

In [1]:
%pip install azure-ai-textanalytics==5.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.6/298.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.


Now that you have created a client for the Text Analytics API, we can use it to detect the language, Evaluate sentiment, Identify key phrases, Extract entities and Extract linked entities from the text. Read the comments in the code below to understand how to use the client to call the API.

In [2]:
!git clone https://github.com/MicrosoftLearning/AI-102-AIEngineer.git

Cloning into 'AI-102-AIEngineer'...
remote: Enumerating objects: 3175, done.
remote: Counting objects: 100% (600/600), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 3175 (delta 423), reused 449 (delta 383), pack-reused 2575
Receiving objects: 100% (3175/3175), 42.48 MiB | 29.33 MiB/s, done.
Resolving deltas: 100% (1770/1770), done.


In [3]:
!pip install python-dotenv

In [4]:
 # import namespaces
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

credential = AzureKeyCredential('595517435adc481fb74616ee20a81160')
cog_client = TextAnalyticsClient(endpoint='https://learn-ai4009.cognitiveservices.azure.com/', credential=credential)

In [6]:
import os
from dotenv import load_dotenv
def main():
    try:
        # Get Configuration Settings
        load_dotenv()
        cog_endpoint = 'https://learn-ai4009.cognitiveservices.azure.com/'
        cog_key = '595517435adc481fb74616ee20a81160'
        # Create client using endpoint and key
        credential = AzureKeyCredential(cog_key)
        cog_client = TextAnalyticsClient(endpoint=cog_endpoint, credential=credential)

        # Analyze each text file in the reviews folder
        reviews_folder = '/content/AI-102-AIEngineer/05-analyze-text/Python/text-analysis/reviews'
        for file_name in os.listdir(reviews_folder):
            # Read the file contents
            print('\n-------------\n' + file_name)
            text = open(os.path.join(reviews_folder, file_name), encoding='utf8').read()
            print('\n' + text)

            # Get language
            detectedLanguage = cog_client.detect_language(documents=[text])[0]
            print('\nLanguage: {}'.format(detectedLanguage.primary_language.name))


            # Get sentiment
            sentimentAnalysis = cog_client.analyze_sentiment(documents=[text])[0]
            print("\nSentiment: {}".format(sentimentAnalysis.sentiment))


            # Get key phrases
            phrases = cog_client.extract_key_phrases(documents=[text])[0].key_phrases
            if len(phrases) > 0:
                print("\nKey Phrases:")
                for phrase in phrases:
                    print('\t{}'.format(phrase))

            # Get entities
            entities = cog_client.recognize_entities(documents=[text])[0].entities
            if len(entities) > 0:
                print("\nEntities")
                for entity in entities:
                    print('\t{} ({})'.format(entity.text, entity.category))


            # Get linked entities
            entities = cog_client.recognize_linked_entities(documents=[text])[0].entities
            if len(entities) > 0:
                print("\nLinks")
                for linked_entity in entities:
                    print('\t{} ({})'.format(linked_entity.name, linked_entity.url))


    except Exception as ex:
        print(ex)

main()


-------------
review5.txt

Un hôtel agréable
L'Hotel Buckingham, Londres, UK
J’adore cet hôtel. Le personnel est très amical et les chambres sont confortables.

Language: French

Sentiment: positive

Key Phrases:
	L'Hotel Buckingham
	agréable
	Londres
	UK
	personnel
	chambres

Entities
	hôtel (Location)
	Hotel Buckingham (Location)
	Londres (Location)
	UK (Location)
	hôtel (Location)
	Le (Quantity)
	personnel (PersonType)
	chambres (Location)

Links
	United Nations (https://en.wikipedia.org/wiki/United_Nations)
	L'Hôtel (https://en.wikipedia.org/wiki/L'Hôtel)
	Buckingham (https://en.wikipedia.org/wiki/Buckingham)
	London (https://en.wikipedia.org/wiki/London)
	United Kingdom (https://en.wikipedia.org/wiki/United_Kingdom)

-------------
review4.txt

Very noisy and rooms are tiny
The Lombard Hotel, San Francisco, USA
9/5/2018
Hotel is located on Lombard street which is a very busy SIX lane street directly off the Golden Gate Bridge. Traffic from early morning until late at night especia

# Translate Text
The Translator service is an Azure AI service that enables you to translate text between languages.

For example, suppose a travel agency wants to examine hotel reviews that have been submitted to the company’s web site, standardizing on English as the language that is used for analysis. By using the Translator service, they can determine the language each review is written in, and if it is not already English, translate it from whatever source language it was written in into English.

The Translator service can automatically detect the source language of text to be translated, but it also enables you to explicitly detect the language in which text is written.

1. In your code, find the GetLanguage function, which currently returns “en” for all text values.
2. In the GetLanguage function, under the comment Use the Translator detect function to find the language of the text.

In [ ]:
from dotenv import load_dotenv
import os
import requests, json

def main():
    global translator_endpoint
    global cog_key
    global cog_region

    try:
        # Get Configuration Settings
        load_dotenv()
        cog_key = 'a4ec489164904aceb320240b7ceafbdf'
        cog_region = 'eastus'
        translator_endpoint = 'https://api.cognitive.microsofttranslator.com/'

        # Analyze each text file in the reviews folder
        reviews_folder = '/content/AI-102-AIEngineer/06-translate-text/Python/text-translation/reviews'
        for file_name in os.listdir(reviews_folder):
            # Read the file contents
            print('\n-------------\n' + file_name)
            text = open(os.path.join(reviews_folder, file_name), encoding='utf8').read()
            print('\n' + text)

            # Detect the language
            language = GetLanguage(text)
            print('Language:',language)

            # Translate if not already English
            if language != 'en':
                translation = Translate(text, language)
                print("\nTranslation:\n{}".format(translation))

    except Exception as ex:
        print(ex)


In [ ]:
def GetLanguage(text):
    # Default language is English
    language = 'en'

    # Use the Translator detect function
    path = '/detect'
    url = translator_endpoint + path

    # Build the request
    params = {
        'api-version': '3.0'
    }

    headers = {
    'Ocp-Apim-Subscription-Key': cog_key,
    'Ocp-Apim-Subscription-Region': cog_region,
    'Content-type': 'application/json'
    }

    body = [{
        'text': text
    }]

    # Send the request and get response
    request = requests.post(url, params=params, headers=headers, json=body)
    response = request.json()

    # Parse JSON array and get language
    language = response[0]["language"]
    # Return the language
    return language


Now that your application can determine the language in which reviews are written, you can use the Translator service to translate any non-English reviews into English.

1. In your code file, find the Translate function, which currently returns and empty string for all text values.
2. In the Translate function, under the comment Use the Translator translate function translates the specified text from its source language into English.

In [ ]:
def Translate(text, source_language):
    translation = ''

    # Use the Translator translate function
    path = '/translate'
    url = translator_endpoint + path

    # Build the request
    params = {
        'api-version': '3.0',
        'from': source_language,
        'to': ['en']
    }

    headers = {
        'Ocp-Apim-Subscription-Key': cog_key,
        'Ocp-Apim-Subscription-Region': cog_region,
        'Content-type': 'application/json'
    }

    body = [{
        'text': text
    }]

    # Send the request and get response
    request = requests.post(url, params=params, headers=headers, json=body)
    response = request.json()

    # Parse JSON array and get translation
    translation = response[0]["translations"][0]["text"]

    # Return the translation
    return translation

main()


-------------
review5.txt

Un hôtel agréable
L'Hotel Buckingham, Londres, UK
J’adore cet hôtel. Le personnel est très amical et les chambres sont confortables.
Language: fr

Translation:
A pleasant hotel
The Hotel Buckingham, London, UK
I love this hotel. The staff is very friendly and the rooms are comfortable.

-------------
review4.txt

Very noisy and rooms are tiny
The Lombard Hotel, San Francisco, USA
9/5/2018
Hotel is located on Lombard street which is a very busy SIX lane street directly off the Golden Gate Bridge. Traffic from early morning until late at night especially on weekends. Noise would not be so bad if rooms were better insulated but they are not. Had to put cotton balls in my ears to be able to sleep--was too tired to enjoy the city the next day. Rooms are TINY. I picked the room because it had two queen size beds--but the room barely had space to fit them. With family of four in the room it was tight. With all that said, rooms are clean and they've made an effort t